In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Map.Strict as M
import Data.List
import Data.Function (on)
import Data.ByteString.UTF8 (toString)

input <- dayLines 7

In [2]:
data Entry = Entry
    { name :: String
    , weight :: Int
    , depends :: [String]
    } deriving Show

parseEntry = do
    name <- toString <$> tillSpace
    A.char8 ' '
    A.char8 '('
    weight <- A.decimal
    A.char8 ')'
    deps <- A.option
        []
        (A.string " -> " *> (map toString <$> A.takeWhile A.isAlpha_ascii `A.sepBy1'` A.string ", "))
    return $ Entry name weight deps    

parsedInput = map (parsed parseEntry) input

In [3]:
head $ map name parsedInput \\ concatMap depends parsedInput

"uownj"

In [4]:
mapping = M.fromList $ zip (map name parsedInput) parsedInput

totalWeight towerName = case mapping M.! towerName of
    entry | null (depends entry) -> weight entry
    Entry _ w ds -> w + sum (map totalWeight ds)

weightsMapping = (map totalWeight . depends) <$> mapping
weightsNameMapping = (\e -> uncurry zip (map totalWeight $ depends e, depends e)) <$> mapping

go :: String -> [String] -> Int
go dep acc = if null (weightsNameMapping M.! dep)
   then error "unreachable"
   else let depWeights = weightsMapping M.! dep in
       if length (nub depWeights) == 1
           then let 
               correct = minimum $ weightsMapping  M.! (head acc)
               in correct - sum depWeights
            else let
                wrong = snd $ maximumBy (compare `on` fst) $ weightsNameMapping M.! dep
                in go wrong (dep:acc)
               
go (head $ map name parsedInput \\ concatMap depends parsedInput) []

596